<a href="https://colab.research.google.com/github/deepakgowtham/EVA4/blob/master/Week4/EVA4_Session_2_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3, padding=1) #28, 28 3
        self.conv2 = nn.Conv2d(8, 16, 3, padding=1) #28, 28, 5
        self.pool1 = nn.MaxPool2d(2, 2)              #28, 14, 6
        self.conv3 = nn.Conv2d(16, 8, 3, padding=1) #14, 14, 10
        self.conv4 = nn.Conv2d(8, 16, 3, padding=1) #14, 14, 14
        self.pool2 = nn.MaxPool2d(2, 2)               #14, 7, 16
        self.conv5 = nn.Conv2d(16, 8, 3) #7, 5, 24
        self.conv6 = nn.Conv2d(8, 16, 3) # 5, 3, 32
        self.conv7 = nn.Conv2d(16, 10, 3) #3, 1, 40
        
        

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = (self.conv7(x))
        
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [0]:
class Net2(nn.Module):
  def __init__(self):
    super(Net2, self).__init__()
    self.conv0 = nn.Conv2d(1, 8, 3, padding=1)
    self.bn1 = nn.BatchNorm2d(8)
    self.drop1=nn.Dropout(0.1)
    self.conv2 = nn.Conv2d(8, 16, 3, padding=1)
    self.conv4 = nn.Conv2d(8, 16, 3, padding=1)
    self.bn2 = nn.BatchNorm2d(16)
    self.drop2=nn.Dropout(0.1)
    self.pool1 = nn.MaxPool2d(2, 2)
    self.bn3 = nn.BatchNorm2d(8)
    self.conv3=nn.Conv2d(16, 32, 3, padding=1)
    self.conv1= nn.Conv2d(8, 8, 3, padding=1)
    self.conv7=nn.Conv2d(10,10,7)
    self.pool = nn.MaxPool2d(2, 2)
    self.bn4 = nn.BatchNorm2d(8)
    self.bn5 = nn.BatchNorm2d(16)
    self.bn8 = nn.BatchNorm2d(8)              
    self.bn16 = nn.BatchNorm2d(16)
    self.bn32 = nn.BatchNorm2d(32)
    self.bn6 = nn.BatchNorm2d(32)
    self.bn64 = nn.BatchNorm2d(64)
    self.bn10= nn.BatchNorm2d(10)
    self.bn11= nn.BatchNorm2d(10)  
    self.conv118=nn.Conv2d(16, 8, 1)
    self.conv1110=nn.Conv2d(32, 10, 1)
    self.gap = nn.AvgPool2d(7)
    self.drop=nn.Dropout(0.1)
    self.drop3=nn.Dropout(0.1)
    self.drop4=nn.Dropout(0.1)
    self.drop5=nn.Dropout(0.1)

  def forward(self, x):
    x=self.drop1(self.bn1((F.relu(self.conv0(x)))))
    x=self.drop2(self.bn2((F.relu(self.conv2(x)))))
    #x=self.drop(self.bn32((F.relu(self.conv3(x)))))
    x=self.bn3(F.relu(self.conv118(self.pool1(x))))
        
    x=self.drop3(self.bn4(F.relu(self.conv1(x))))
    x=self.drop4(self.bn5((F.relu(self.conv4(x)))))
    x=self.drop5(self.bn6((F.relu(self.conv3(x)))))
    x=self.bn11(self.conv7(self.bn10(F.relu(self.conv1110(self.pool(x))))))
    x = x.view(-1, 10)
    return F.log_softmax(x)


In [4]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net2().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
       BatchNorm2d-2            [-1, 8, 28, 28]              16
           Dropout-3            [-1, 8, 28, 28]               0
            Conv2d-4           [-1, 16, 28, 28]           1,168
       BatchNorm2d-5           [-1, 16, 28, 28]              32
           Dropout-6           [-1, 16, 28, 28]               0
         MaxPool2d-7           [-1, 16, 14, 14]               0
            Conv2d-8            [-1, 8, 14, 14]             136
       BatchNorm2d-9            [-1, 8, 14, 14]              16
           Conv2d-10            [-1, 8, 14, 14]             584
      BatchNorm2d-11            [-1, 8, 14, 14]              16
          Dropout-12            [-1, 8, 14, 14]               0
           Conv2d-13           [-1, 16, 14, 14]           1,168
      BatchNorm2d-14           [-1, 16,

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:45: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:


torch.manual_seed(1)
batch_size = 64

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            #optimizer.zero_grad()
            output = model(data)
            
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.4f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [7]:

model = Net2().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1)
scheduler = StepLR(optimizer, step_size=1, gamma=0.83)

for epoch in range(1, 21):
    print(f'Epoch: {epoch}','LR:', scheduler.get_lr())
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

  0%|          | 0/938 [00:00<?, ?it/s]

Epoch: 1 LR: [0.1]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:45: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0674, Accuracy: 9849/10000 (98.4900%)

Epoch: 2 LR: [0.06889]


  0%|          | 0/938 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0227, Accuracy: 9940/10000 (99.4000%)

Epoch: 19 LR: [0.0029005727002779317]


  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0221, Accuracy: 9938/10000 (99.3800%)

Epoch: 20 LR: [0.002407475341230683]


loss=0.06719952821731567 batch_id=937: 100%|██████████| 938/938 [00:24<00:00, 38.20it/s]



Test set: Average loss: 0.0241, Accuracy: 9933/10000 (99.3300%)



In [0]:
9903/10000

0.9903

In [0]:
model=None